# Replication of arxiv:1503.01007

This notebook replicates this [article](http://arxiv.org/abs/1503.01007)

In [1]:
import os
import json

import numpy as np

import pandas as pd
import tqdm

We'll definitely need to plot something

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

Uninterruptible section

In [3]:
import signal

class DelayedKeyboardInterrupt(object):
    def __enter__(self):
        self.signal_received = False
        self.old_handler = signal.getsignal(signal.SIGINT)
        signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        self.signal_received = (sig, frame)

    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
        if self.signal_received:
            self.old_handler(*self.signal_received)

Fix the random seed

In [4]:
random_state = np.random.RandomState(0x0BADC0DE)

Import Theano and Lasagne

In [5]:
# %env THEANO_FLAGS='device=cuda0,force_device=True,mode=FAST_RUN,floatX=float32'

import theano
theano.config.exception_verbosity = 'high'

import theano.tensor as tt

import lasagne
from lasagne.utils import floatX

Using cuDNN version 5103 on context None
Mapped name None to device cuda: GeForce GTX 980 Ti (0000:06:00.0)


Fix Lasagne's random seed.

In [6]:
lasagne.random.set_rng(np.random.RandomState(0xDEADC0DE))

Build the vocabulary

In [7]:
## It is very important that these service characters be added first
vocab = [">", "<", "a", "b", "c"]
token_to_index = {w: i for i, w in enumerate(vocab)}

A function to lines into character id vectors.

In [8]:
def as_matrix(lines, max_len=None):
    if isinstance(lines, str):
        lines = [lines]

    max_len = max_len or max(map(len, lines))
    matrix = np.full((len(lines), max_len), -1, dtype='int32')

    for i, line in enumerate(lines):
        row_ix = [token_to_index.get(c, -1)
                  for c in line[:max_len]]

        matrix[i, :len(row_ix)] = row_ix

    return matrix

A function to sample a batch from History-Reply pairs

In [9]:
def generate_batch(batch_size=32, max_seq_len=None):
    sequences = []
    for i in range(batch_size):
        n, m = random_state.randint(1, 10, size=(2,))
        sequences.append(">" + "a" * n + "b" * m + "c" * (n + m) + "<")
    return as_matrix(sequences, max_seq_len)

Define a simple seq2seq network (preferably in Lasagne)

In [10]:
from lasagne.layers import InputLayer, EmbeddingLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import SliceLayer

from GRUwithStack import GRUStackLayer, GRUStackReadoutLayer
from broadcast import BroadcastLayer, UnbroadcastLayer

The architecture hyper parameters

In [11]:
n_embed_char, n_hidden_decoder = len(vocab), 32
n_stack_depth, n_stack_width = 50, 3
n_recurrent_layers = 1

### Helpers

A helper to create stacked RNN

In [12]:
from lasagne.layers.base import Layer

def gru_column(input, stack, num_units, hidden=None, **kwargs):
    kwargs.pop("only_return_final", None)
    if not isinstance(hidden, (list, tuple)):
        hidden = len(stack) * [None]

    name, column = kwargs.pop("name", "default"), [input]
    for i, (l_stack, l_hidden) in enumerate(zip(stack, hidden)):
        kwargs_ = kwargs.copy()
        if isinstance(l_hidden, Layer):
            kwargs_.pop("learn_init", None)
            kwargs_["hid_init"] = l_hidden

        layer = GRUStackLayer(column[-1], l_stack, num_units,
                              name=os.path.join(name, "grustack_%02d" % i),
                              **kwargs_)
        column.append(layer)
    return column[1:]

Create readouts of the last hidden state

In [13]:
def gru_hidden_readout(column, indices):
    hidden = []
    for layer in column:
        name = os.path.join(layer.name, "slice")
        slice_ = SliceLayer(layer, indices, axis=1, name=name)
        hidden.append(slice_)
    return hidden

A function ot get the last state of the stack

In [14]:
def gru_stack_readout(column, indices):
    state = []
    for layer in column:
        name =  os.path.join(layer.name, "stack")
        stack  = GRUStackReadoutLayer(layer, name=name)
        slice_ = SliceLayer(stack, indices, axis=1,
                            name=os.path.join(name, "slice"))
        state.append(slice_)
    return state

### Encoder-decoder

Tap into the common embedding layer but with decoder's own input.

In [15]:
l_decoder_mask = InputLayer((None, None), name="decoder/mask")
l_decoder_embed = InputLayer((None, None, n_embed_char), name="decoder/input")

Construct stack inputs

In [16]:
stacks = [InputLayer((None, n_stack_depth, n_stack_width),
                     name="encoder/stack%02d" % i)
          for i in range(n_recurrent_layers)]

Construct layers of GRU-s which recieve the final state of the encoder's network.

In [17]:
dec_rnn_layers = gru_column(l_decoder_embed, stacks, n_hidden_decoder,
                            hidden=None, mask_input=l_decoder_mask,
                            learn_init=False, backwards=False,
                            name="decoder")

dec_rnn_layers_sliced = gru_hidden_readout(dec_rnn_layers, -1)
dec_rnn_layers_stack = gru_stack_readout(dec_rnn_layers, -1)

<img src="http://s32.postimg.org/vefrk7vqt/stack_gru.png"/>

Read the output of the top layer of the RNN and re-embed into the character space

In [18]:
l_decoder_reembedder = DenseLayer(dec_rnn_layers[-1], num_units=len(vocab),
                                  nonlinearity=None, num_leading_axes=2,
                                  name="decoder/project")

Construct the softmax layer

In [19]:
l_bc = BroadcastLayer(l_decoder_reembedder, broadcasted_axes=(0, 1), name="decoder/bc")
l_softmax = NonlinearityLayer(l_bc, nonlinearity=lasagne.nonlinearities.softmax, name="decoder/softmax")
l_decoder_output = UnbroadcastLayer(l_softmax, l_bc, name="decoder/ub")

### Embedding subgraph (pinkish)

The common embedding layer

In [20]:
l_input_char = InputLayer((None, None), name="char/input")
l_embed_char = EmbeddingLayer(l_input_char, len(vocab), n_embed_char, name="char/embed")

### Loss

In [21]:
v_decoder_input = tt.imatrix(name="decoder/input")
v_decoder_embed = lasagne.layers.get_output(l_embed_char, v_decoder_input)

inputs = {l_decoder_embed: v_decoder_embed,
          l_decoder_mask: tt.ge(v_decoder_input, 0)}

The initial stack states (zero)

In [22]:
for layer in dec_rnn_layers:
    stack = layer.input_layers[1]
    inputs[stack] = tt.zeros((v_decoder_embed.shape[0], n_stack_depth, n_stack_width),
                             dtype=theano.config.floatX)

Get the output of the decoder

In [23]:
v_decoder_output, v_decoder_mask = lasagne.layers.get_output(
    [l_decoder_output, l_decoder_mask], inputs, deterministic=False)

Slice the output to match the forward character-level language model

In [24]:
v_predicted = v_decoder_output[:, :-1].reshape(
    (-1, v_decoder_output.shape[-1]))

v_targets = v_decoder_input[:, 1:].reshape((-1,))

v_mask = v_decoder_mask[:, 1:].reshape((-1,))

Construct the cross-entropy loss

In [25]:
loss_ij = lasagne.objectives.categorical_crossentropy(v_predicted, v_targets)
loss = (loss_ij * v_mask).sum()
loss /= v_mask.sum()

Add $l^2$ regularization.

In [26]:
# from lasagne.regularization import regularize_network_params, l2

# reg_l2 = regularize_network_params(l_decoder_output, l2) * 10e-5

# loss += reg_l2

Collect all trainable parameters 

In [27]:
trainable = []
trainable.extend(lasagne.layers.get_all_params(l_embed_char, trainable=True))
trainable.extend(lasagne.layers.get_all_params(l_decoder_output, trainable=True))

Get the updates

In [28]:
learning_rate = theano.shared(floatX(1e-3), name="eta")
updates = lasagne.updates.adam(loss, trainable, learning_rate)

Create the ops

In [29]:
op_train = theano.function([v_decoder_input], loss,
                           updates=updates, givens={},
                           mode=theano.Mode(optimizer="fast_run"))

In [32]:

v_decoder_logits = lasagne.layers.get_output(l_decoder_reembedder, inputs, deterministic=True)

op_predict_logits = theano.function([v_decoder_input],
                                     v_decoder_logits, mode=theano.Mode(optimizer="fast_run"))

### The generator

A handy slicer (copied and modified)

In [33]:
def slice_(x, i, n):
    s = x[..., slice(i, i + n)]
    return s if n > 1 else tt.addbroadcast(s, -1)

Define one step of the scan function

In [34]:
# Generator's one step update function
def generator_step_sm(x_tm1, h_tm1, m_tm1, s_tm1, tau, eps):
    """One step of the generative decoder version."""
    # x_tm1 is `BxT` one-hot, h_tm1 is `batch x ...`
    # m_tm1 is `batch`, tau, eps are scalars

    # collect the inputs
    inputs = {l_decoder_embed: x_tm1.dimshuffle(0, "x", 1),
              l_decoder_mask: m_tm1.dimshuffle(0, "x")}

    # Connect the prev variables to the the hidden and stack state feeds
    j = 0
    for layer in dec_rnn_layers:
        inputs[layer.hid_init] = slice_(h_tm1, j, layer.num_units)
        j += layer.num_units

    j = 0
    for layer in dec_rnn_layers:
        layer = layer.input_layers[1]
        dep, wid = layer.output_shape[-2:]
        stack_slice_ = slice_(s_tm1, j, dep * wid)
        inputs[layer] = stack_slice_.reshape((-1, dep, wid))
        j += dep * wid

    # Get the outputs
    outputs = [l_decoder_reembedder]
    for pair in zip(dec_rnn_layers_sliced, dec_rnn_layers_stack):
        outputs.extend(pair)

    # propagate through the decoder column
    logit_t, *rest = lasagne.layers.get_output(outputs, inputs,
                                               deterministic=True)
    h_t_list, s_t_list = rest[::2], rest[1::2]

    # Pack the hidden and flattened stack states
    h_t = tt.concatenate(h_t_list, axis=-1)
    s_t = tt.concatenate([v.flatten(ndim=2) for v in s_t_list], axis=-1)
    
    # Generate the next symbol: logit_t is `Bx1xV`
    logit_t = logit_t[:, 0]
    prob_t = tt.nnet.softmax(logit_t)

    # Gumbel-softmax sampling: Gumbel (e^{-e^{-x}}) distributed random noise
    gumbel = -tt.log(-tt.log(theano_random_state.uniform(size=logit_t.shape) + eps) + eps)
#     logit_t = theano.ifelse.ifelse(tt.gt(tau, 0), gumbel + logit_t, logit_t)
#     inv_temp = theano.ifelse.ifelse(tt.gt(tau, 0), 1.0 / tau, tt.constant(1.0))
    logit_t = tt.switch(tt.gt(tau, 0), gumbel + logit_t, logit_t)
    inv_temp = tt.switch(tt.gt(tau, 0), 1.0 / tau, tt.constant(1.0))

    # Get the softmax: x_t is `BxV`
    x_t = tt.nnet.softmax(logit_t * inv_temp)

    # Get the best symbol
    c_t = tt.cast(tt.argmax(x_t, axis=-1), "int8")

    # Get the estimated probability of the picked symbol.
    p_t = prob_t[tt.arange(c_t.shape[0]), c_t]

    # Compute the mask and inhibit the propagation on a stop symbol.
    # Recurrent layers return the previous state if m_tm1 is Fasle
    m_t = m_tm1 & tt.gt(c_t, vocab.index("<"))
    c_t = tt.switch(m_t, c_t, vocab.index("<"))

    # There is no need to freeze the states as they will be frozen by
    # the RNN passthrough according to the mask `m_t`.

    # Embed the current character.
    x_t = tt.dot(x_t, l_embed_char.W)

    return x_t, h_t, m_t, s_t, p_t, c_t

Create scalar inputs to the scan loop. Also initialize the random stream.

In [35]:
theano_random_state = tt.shared_randomstreams.RandomStreams(seed=42)

eps = tt.fscalar("generator/epsilon")
n_steps = tt.iscalar("generator/n_steps")
tau = tt.fscalar("generator/gumbel/tau")

n_batch = tt.iscalar("generator/n_batch")

Helper functions to freeze the GRULayer's hidden input's initialization, if one is a parameter.

In [36]:
def GRULayer_freeze(layer, n_batch):
    assert isinstance(layer, GRUStackLayer)
    if isinstance(layer.hid_init, Layer):
        return layer

    assert not (layer.hid_init_incoming_index > 0)
    assert isinstance(layer.hid_init, theano.compile.SharedVariable)

    # Broadcast the fixed /learnt hidden init over the batch dimension
    hid_init = tt.dot(tt.ones((n_batch, 1)), layer.hid_init)

    # Create a fake Input Layer, which receives it as input
    layer._old_hid_init = layer.hid_init
    layer.hid_init = InputLayer((None, None), input_var=hid_init,
                                name=os.path.join(layer.name,
                                                  "hid_init_fix"))
    
    # Cache former values
    layer._old_input_layers = layer.input_layers
    layer._old_input_shapes = layer.input_shapes
    layer._old_hid_init_incoming_index = layer.hid_init_incoming_index
    
    # Emulate hidden layer input (is in GRULayer/MergeLayer.__init__())
    layer.input_layers.append(layer.hid_init)
    layer.input_shapes.append(layer.hid_init.output_shape)
    layer.hid_init_incoming_index = len(layer.input_layers) - 1

    layer._layer_frozen = True
    return layer

Freeze the hidden inputs of the decoder layers, which do not tap into the encoder.

In [37]:
for layer in dec_rnn_layers:
    GRULayer_freeze(layer, n_batch)

Readout the last state of the stack (per dialogue in the batch) from the encoder.

In [38]:
dec_stack_inits = [tt.zeros((n_batch, n_stack_depth, n_stack_width))
                   for l in dec_rnn_layers]

dec_hid_inits = [tt.zeros((n_batch, n_hidden_decoder))
                 for l in dec_rnn_layers]

Prepare the initial values.

In [39]:
h_0 = tt.concatenate(dec_hid_inits, axis=-1)
s_0 = tt.concatenate([v.flatten(ndim=2) for v in dec_stack_inits], axis=-1)

x_0 = tt.fill(tt.zeros((n_batch,), dtype="int32"), vocab.index(">"))
x_0 = lasagne.layers.get_output(l_embed_char, x_0)

m_0 = tt.ones((n_batch,), 'bool')

Add a scan op and compile

In [40]:
result, updates = theano.scan(generator_step_sm, sequences=None, n_steps=n_steps,
                              outputs_info=[x_0, h_0, m_0, s_0, None, None], return_list=True,
                              non_sequences=[tau, eps], strict=False,
                              go_backwards=False, name="generator/scan")
x_t, h_t, m_t, s_t, p_t, c_t = [r.swapaxes(0, 1) for r in result]

compile_mode = theano.Mode(optimizer="fast_run", linker="cvm")
op_generate = theano.function([n_batch, n_steps, tau], [c_t, h_t, m_t, s_t, p_t],
                              updates=updates, givens={eps: floatX(1e-20)},
                              mode=compile_mode)

A generator procedure, which automatically select the best replies (shortest).

In [41]:
def generate(n_batches, n_steps, tau=0):
    x_t, h_t, m_t, s_t, p_t = op_generate(n_batches, n_steps, tau)

    # may produce NaN, but they are shifted in the back by arsort
    perplexity, n_chars = (- np.log2(p_t) * m_t).sum(axis=-1), m_t.sum(axis=-1)
    perplexity /= n_chars

    result = []
    for i in perplexity.argsort():
        reply = "".join(map(vocab.__getitem__, x_t[i, :n_chars[i]]))
        result.append(reply)
        # result.append((reply, perplexity[i]))  # , s_t[i, -1]))

    return result

<br/>

### The main training loop

In [42]:
def sample_qa():
    replies = generate(10, 140, tau=2**-15)
    tqdm.tqdm.write(", ".join(replies))

Reset the history

In [43]:
epoch, loss_val_hist = 0, []

Now let's train the shit!

In [44]:
batch_size, n_epochs = 128, 5000
progress_fmt_ = "%(loss).3f"
with tqdm.tqdm(total=n_epochs-epoch) as progress_:
    while epoch < n_epochs:
        if (epoch % 50) == 0:
            sample_qa()

        batch = generate_batch(batch_size, max_seq_len=512)
        with DelayedKeyboardInterrupt():
            loss_val_hist.append(op_train(batch))

        progress_.postfix = progress_fmt_ % {
            "loss": np.mean(loss_val_hist[-100:]),
        }
        progress_.update(1)
        epoch += 1

  0%|          | 0/5000 [00:00<?, ?it/s]

ccbc, c, cc, cac, bcab, acbccc, a, , , 


  1%|          | 50/5000 [00:21<34:53,  2.36it/s, 1.366]

bbc, c, c, c, b, , , , , 


  2%|▏         | 100/5000 [00:42<34:31,  2.37it/s, 1.184]

...

 98%|█████████▊| 4900/5000 [34:40<00:42,  2.37it/s, 0.212]

aaaaaaaaabbbbbbccccccccccccccc, aaaaaaaabbbbbbbbbccccccccccccccccc, aaaaaaabbbbbbccccccccccccc, aaaaaaaaabbbbccccccccccccc, aaaaaaaaabbbcccccccccccc, aaaabbbbbbbccccccccccc, aaaaabbbbbcccccccccc, aabbbbbbbccccccccc, aabbbbbbbccccccccc, abbbbbbccccccc


 99%|█████████▉| 4950/5000 [35:01<00:21,  2.37it/s, 0.211]

aaaaaaaabbbbcccccccccccc, aaaaaaaaabbbcccccccccccc, aaabbbbbbbbccccccccccc, aaaaaaaabbbccccccccccc, aaaaabbbbbcccccccccc, aabbbbbbbccccccccc, aaaaaabbbccccccccc, aaaabbbbcccccccc, aaabbbbccccccc, aaabbbcccccc


100%|██████████| 5000/5000 [35:23<00:00,  2.37it/s, 0.211]


In [45]:
generate(10, 150, tau=1e-3)

['aaabbbbbbbbccccccccccc',
 'aaaaaaabbbbccccccccccc',
 'aabbbbbbbbbccccccccccc',
 'abbbbbbbbbcccccccccc',
 'aaaaaaabcccccccc',
 'aaaaaabbcccccccc',
 'aaaaaabbcccccccc',
 'abbbbbbbcccccccc',
 'aabbbbbccccccc',
 'aabccc']